In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'data/ds_salaries.csv')

### PREPROCESAMIENTO

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [4]:
df.columns #saber las columnas

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [5]:
df.shape #saber las dimensiones

(3755, 11)

In [6]:
df.dtypes #saber los tipos de datos

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [7]:
df.isnull().sum() #saber los valores nulos

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [8]:
df.duplicated().sum() #saber los valores duplicados

1171

In [9]:
def mas_repetidos(df):
    df_duplicados = df[df.duplicated(keep=False)]
    df_agrupado = df_duplicados.groupby(df.columns.tolist()).size().reset_index().rename(columns={0: 'count'})
    df_ordenado = df_agrupado.sort_values(by='count', ascending=False)
    return df_ordenado

mas_repetidos(df)


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,count
223,2022,SE,FT,Data Scientist,191475,USD,191475,US,100,US,M,21
201,2022,SE,FT,Data Scientist,141525,USD,141525,US,100,US,M,21
441,2023,SE,FT,Data Engineer,252000,USD,252000,US,0,US,M,13
399,2023,SE,FT,Data Engineer,129000,USD,129000,US,0,US,M,13
221,2022,SE,FT,Data Scientist,185900,USD,185900,US,0,US,M,12
...,...,...,...,...,...,...,...,...,...,...,...,...
229,2022,SE,FT,Data Scientist,230000,USD,230000,US,100,US,M,2
226,2022,SE,FT,Data Scientist,207000,USD,207000,US,100,US,M,2
225,2022,SE,FT,Data Scientist,203500,USD,203500,US,0,US,M,2
224,2022,SE,FT,Data Scientist,198440,USD,198440,US,0,US,L,2


In [10]:
df.drop_duplicates(inplace=True) #eliminar duplicados

In [11]:
df.shape

(2584, 11)

### EDA

In [12]:
df.head(1)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L


en que pais estan las compañias mas grandes
 

In [13]:
paises=[]
for i in range(len(df)):
    if df.iloc[i]['company_size']=='L':
        paises.append(((df.iloc[i]['company_location'])))

(pd.DataFrame(paises).value_counts().head(5))

0 
US    220
IN     37
GB     20
CA     18
DE     16
Name: count, dtype: int64

En que puesto se gana mas

In [15]:
df.groupby(['job_title']).agg({'salary_in_usd':'mean'}).round(2).sort_values(by='salary_in_usd',ascending=False).head()

,salary_in_usd
job_title,
Data Science Tech Lead,375000.00
Cloud Data Architect,250000.00
Data Lead,212500.00
Data Analytics Lead,211254.50
Principal Data Scientist,198171.12


pais de residencia donde mas se cobra?

In [32]:
df.groupby(['employee_residence']).agg({'salary_in_usd':'sum'}).round(2).sort_values(by='salary_in_usd',ascending=False).head()


,salary_in_usd
employee_residence,
US,291469387
GB,13173136
CA,10599647
DE,4458334
ES,2957876


tipo de trabajo donde mas se cobra

In [33]:
df.groupby(['employment_type']).agg({'salary_in_usd':'mean'}).round(2).sort_values(by='salary_in_usd',ascending=False)

,salary_in_usd
employment_type,
FT,134434.61
CT,113446.90
FL,51807.80
PT,39533.71


¿El nivel de  experiencia es proporcional al salario 
 

cuanto se cobra de media por cada nivel de experiencia?
 

In [19]:
df.groupby(['experience_level']).agg({'salary_in_usd':'mean'}).round(2).sort_values(by='salary_in_usd',ascending=False)

,salary_in_usd
experience_level,
EX,191078.21
SE,153897.44
MI,101828.78
EN,72648.69


en qué tamaño de compañía trabajan más

In [27]:
df['company_size'].value_counts().sort_values(ascending=False)

company_size
M    2028
L     409
S     147
Name: count, dtype: int64

en que tamaño de compañias trabajan mas 'Data Engineer'

In [58]:
df[df['job_title']=='Data Engineer'].groupby('company_size')['job_title'].value_counts()


company_size  job_title    
L             Data Engineer     67
M             Data Engineer    523
S             Data Engineer      8
Name: count, dtype: int64

en que pais se cobra  mas por tipo de empleo?
 

In [72]:
df.groupby(['company_location','job_title']).agg({'salary_in_usd':'mean'}).round(2).sort_values(by=['salary_in_usd','job_title'],ascending=False)

salary_in_usd
company_location job_title                                   
IL               AI Scientist                        423834.0
US               Data Analytics Lead                 405000.0
                 Data Science Tech Lead              375000.0
IN               AI Developer                        300000.0
US               Director of Data Science            294375.0
...                                                       ...
CH               Big Data Engineer                     5882.0
IN               Data Science Consultant               5707.0
                 Power BI Developer                    5409.0
                 3D Computer Vision Researcher         5409.0
CZ               NLP Engineer                          5132.0

[356 rows x 1 columns]

wich are experience level more required

In [31]:
df['experience_level'].value_counts()

experience_level
SE    1554
MI     664
EN     270
EX      96
Name: count, dtype: int64

cuáles se repiten más en este set de datos, las empresas medianas, grandes o pequeñas